# **Juypter "GUI" using ipywidgets**

helpful links:

https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916

https://towardsdatascience.com/interactive-spreadsheets-in-jupyter-32ab6ec0f4ff

In [ ]:
import ipywidgets as widgets

import pandas as pd
import numpy as np

## **Filtering a list**
taken from https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916

In [ ]:
url = r"https://data.london.gov.uk/download/number-international-visitors-london/b1e0f953-4c8a-4b45-95f5-e0d143d5641e/international-visitors-london-raw.csv"
df_london = pd.read_csv(url, encoding = "ISO-8859-1")

In [ ]:
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [ ]:
dropdown_year = widgets.Dropdown(options=unique_sorted_values_plus_ALL(df_london.year))

output_year = widgets.Output()

def dropdown_year_eventhandler(change):
    output_year.clear_output()        
    with output_year:
        if (change.new == ALL):
            display(df_london)
        else:
            display(df_london[df_london.year == change.new])
        
dropdown_year.observe(dropdown_year_eventhandler, names='value')
display(dropdown_year)

In [ ]:
display(output_year)

-----
# **Select file using a button and askopenfilename**

code from: https://codereview.stackexchange.com/questions/162920/file-selection-button-for-jupyter-notebook
        
small bugfix if no file selected

In [ ]:
import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog


class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        _files =filedialog.askopenfilename(multiple=True)
        if _files:
            b.files = _files

            b.description = "Files Selected"
            b.icon = "check-square-o"
            b.style.button_color = "green"
        else:
            b.files = []
            b.description = "File selection aborded"
            b.icon = "square-o"
            b.style.button_color = "red"

In [ ]:
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

In [ ]:
# In a different cell of the same Jupyter Notebook You can access the file list by using the following:
my_button.files

---
## **Simple Slider using interact**

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt

In [ ]:
output_vector_plot = widgets.Output()

@widgets.interact(mag1=(0, 100), ang1=(0, 360), mag2=(0, 100), ang2=(0, 360))
def plot_vectors(mag1=1, ang1=0, mag2=1, ang2=90):
    with output_vector_plot:
        output_vector_plot.clear_output()
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        vec1 = [np.cos(ang1*np.pi/180)*mag1,np.sin(ang1*np.pi/180)*mag1]
        vec2 = [np.cos(ang2*np.pi/180)*mag2,np.sin(ang2*np.pi/180)*mag1]

        ax.arrow(0,0, vec1[0], vec1[1], width=0.01, color='blue')
        ax.arrow(0,0,vec2[0], vec2[1], width=0.01, color='red')
        plt.axis('square')
        plt.draw()
        
        dot_product = np.dot(vec1/np.linalg.norm(vec1), vec2/np.linalg.norm(vec2))
        print(dot_product)
        angle = np.arccos(dot_product)
        print(f'Angle between: {angle*180/np.pi:3.1f} °')
#interact(plot_vectors, mag1=w_mag1.value, ang1=w_ang1.value);
display(output_vector_plot)

following is from https://kapernikov.com/ipywidgets-with-matplotlib/

In [ ]:
def say_my_name(name):
    """
    Print the current widget value in short sentence
    """
    print(f'My name is {name}')
     
widgets.interact(say_my_name, name=["Jim", "Emma", "Bond"]);

---
# **Designing an electrical field**
Dashboard example

In [ ]:
import numpy as np
def elec_field(x, y, x_coords, y_coords, charges):
    """Calculates the electrical field based on two charges"""
    # this function was taken from https://gist.github.com/alexlib/bac36d4c0db9f28fb4b4dbe7023b705d
    Ex = np.zeros_like(x)
    Ey = np.zeros_like(x)
    for x0, y0, q in zip(x_coords, y_coords, charges):
        R = np.sqrt((x - x0)**2 + (y - y0)**2) + 1e-6
        Ex += q*(x - x0)/R**3
        Ey += q*(y - y0)/R**3
    return Ex, Ey


def build_efield(x0, y0, x1, y1):
    y, x = np.mgrid[-3:3:21j, -3:3:21j] # coordinate grid
    _E1 = elec_field(x, y, [x0, x1], [0, 0], [1, -1])
    E1 = np.stack(_E1, axis=-1)
    return E1

In [ ]:
source_slider1 = widgets.FloatSlider(min=-1, max=0, value=-0.5, description='x0')
source_slider2 = widgets.FloatSlider(min=0, max=1, value=0.5, description='x1')

btn_apply = widgets.Button(description='Apply')
def btn_eventhandler(obj):
    with output_efield:
        output_efield.clear_output()
        x0 = source_slider1.value
        x1 = source_slider2.value
        E = build_efield(x0, 0, x1, 0)
        Emag = np.linalg.norm(E, axis=-1)
        plt.figure()
        plt.contourf(x, y, np.log10(Emag), cmap="autumn")
        plt.show()
    
btn_apply.on_click(btn_eventhandler)

output_efield = widgets.Output()

_hbox = widgets.HBox([source_slider1, source_slider2])
_vbox = widgets.VBox([_hbox, btn_apply])


tab = widgets.Tab([_vbox, output_efield])
tab.set_title(0, 'Settings')
tab.set_title(1, 'Plot')

Plot is shown in second tab. Not user friendly but fair enough to show the usage of tabs

In [ ]:
display(tab)